# Exercise 4
## Match searches with bookings

- For every search in the searches file, find out whether
the search ended up in a booking or not (using the info
in the bookings file). For instance, search and booking
origin and destination should match.
- For the bookings file, origin and destination are the
columns dep_port and arr_port, respectively.
- Generate a CSV file with the search data, and an
additional field, containing 1 if the search ended up in
a booking, and 0 otherwise.

In [1]:
import pandas as pd

In [2]:
import bz2
import os

Let's see which columns I can use to do this...

In [10]:
bookings_n1000 = pd.read_csv(r"C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge\bookings.csv.bz2",
                             compression= "bz2", sep = "^", error_bad_lines = False, memory_map = True, nrows=1000)

In [11]:
bookings_n1000.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [17]:
bookings_n1000.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [12]:
searches_n1000 = pd.read_csv(r"C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge\searches.csv.bz2",
                             compression= "bz2", sep = "^", error_bad_lines = False, memory_map = True, nrows=1000)

In [13]:
searches_n1000.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


I think I will use the Origin and Destination columns and the date/time info to match both files

Let's try and do this in chunks too. Firstly, I'll look for duplicates and I'll normalise both datasets. I'll save the new datasets without duplicates in a different file

In [40]:
%%time
path_searches_without_duplicates = "C:/Users/paula/Desktop/Paula/Máster/9. Data Science challenge/challenge/searches_without_duplicates.csv.bz2"

chunksize = 400000
searches_chunks = pd.read_csv(r"C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge\searches.csv.bz2",
                              compression= "bz2", 
                              sep = "^",
                              error_bad_lines = False,
                              memory_map = True,
                              usecols = ["Origin", "Destination", "Date", "Time"],
                              chunksize= chunksize, dtype= str, low_memory= False)

all_chunks = pd.DataFrame()


for i, chunk in enumerate(searches_chunks):
    
    all_chunks = all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace = True)

all_chunks["Destination"] = all_chunks["Destination"].str.strip().str.upper()
all_chunks["Origin"] = all_chunks["Origin"].str.strip().str.upper()
            
all_chunks.to_csv(path_searches_without_duplicates, sep ="^", index = False)    
print(len(all_chunks))
    
  


358998
Wall time: 5min 6s


Removing duplicates, the searches files now has 358.998 rows, instead of the 20.390.199 initial ones...

As we can see above, in the bookings file, there are columns name with spaces. One of them is: "act_date           "
As this is one of the columns we need, for the bookings file I won't use the "usecols" parameter when reading the csv and I'll delete spaces in the column's name too 

In [84]:
%%time
path_bookings_without_duplicates = "C:/Users/paula/Desktop/Paula/Máster/9. Data Science challenge/challenge/bookings_without_duplicates.csv.bz2"

chunksize = 400000
bookings_chunks = pd.read_csv(r"C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge\bookings.csv.bz2",
                              compression= "bz2", 
                              sep = "^",
                              error_bad_lines = False,
                              memory_map = True,
                              chunksize= chunksize, dtype= str, low_memory= False)

all_chunks = pd.DataFrame()


for i, chunk in enumerate(bookings_chunks):
    
    all_chunks = all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace = True)

all_chunks.columns = all_chunks.columns.str.strip()
            
all_chunks.to_csv(path_bookings_without_duplicates, sep ="^", index = False)    
print(len(all_chunks))

1000003
Wall time: 10min 5s


Removing duplicates, the bookings files now has 1.000.003 rows, instead of the 10.000.011 initial ones

In [85]:
bookings_nd_n1000 = pd.read_csv(r"C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge\bookings_without_duplicates.csv.bz2",
                             compression= "bz2", sep = "^", error_bad_lines = False, memory_map = True, nrows=1000)

In [86]:
bookings_nd_n1000.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [87]:
bookings_nd_n1000.columns

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

In [88]:
bookings_nd = pd.read_csv(r"C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge\bookings_without_duplicates.csv.bz2",
                             compression= "bz2", sep = "^", error_bad_lines = False, memory_map = True, dtype= str,
                          usecols= ["act_date", "dep_port", "arr_port"])

In [89]:
searches_nd = pd.read_csv(r"C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge\searches_without_duplicates.csv.bz2",
                             compression= "bz2", sep = "^", error_bad_lines = False, memory_map = True, dtype= str)

Now, I need to match the searchings with the bookings, to check if it ended up being a booking or not. Therefore, I'll use the date and the origin and destination airport info. Firstly, let's make sure the date is an appropiate date format in both files.
Also, let's make sure there are no spaces or weird formats in the other columns either

In [90]:
bookings_nd["arr_port"].unique()

array(['LHR     ', 'CLT     ', 'SVO     ', ..., 'VEL     ', 'SLM     ',
       'SG      '], dtype=object)

In [91]:
len_s_dest = searches_nd["Destination"].str.len().mean()
len_s_dest

2.999997214461402

In [92]:
bookings_nd["arr_port"] = bookings_nd["arr_port"].str.strip(" ")
bookings_nd["dep_port"] = bookings_nd["dep_port"].str.strip(" ")

In [93]:
len_b_dest = bookings_nd["dep_port"].str.len().mean()
len_b_dest

2.999999000003

In [94]:
len_b_arr = bookings_nd["arr_port"].str.len().mean()
len_b_arr

2.999999000003

In [95]:
import numpy as np

Let's delete first the rows with wrong IATA codes (lenght lower than 3 characters)

In [96]:
#Wrong lenght in the arrival airport code in the bookings file
wrong_len_b_arr = bookings_nd["arr_port"].str.len() < 3
wrong_len_b_arr

0          False
1          False
2          False
3          False
4          False
           ...  
999998     False
999999     False
1000000    False
1000001     True
1000002    False
Name: arr_port, Length: 1000003, dtype: bool

In [97]:
np.count_nonzero(wrong_len_b_arr)

1

In [98]:
bookings_nd = bookings_nd.drop([1000001])

In [99]:
wrong_len_b_arr = bookings_nd["arr_port"].str.len() < 3
wrong_len_b_arr
np.count_nonzero(wrong_len_b_arr)

0

In [100]:
#Wrong lenght in the origin airport code in the bookings file
wrong_len_b_dep = bookings_nd["dep_port"].str.len() < 3
wrong_len_b_dep.sort_values(ascending= False)

1000002    False
333343     False
333341     False
333340     False
333339     False
           ...  
666663     False
666662     False
666661     False
666660     False
0          False
Name: dep_port, Length: 1000002, dtype: bool

In [101]:
np.count_nonzero(wrong_len_b_dep)

0

It seems that the row with wrong Arrival IATA code was the same one with wrong destination IATA code, let's do the same in the searches file

In [102]:
#Wrong lenght in the arrival airport code in the searches file
wrong_len_s_dest = searches_nd["Destination"].str.len() < 3
wrong_len_s_dest.sort_values(ascending= False)

358997     True
119668    False
119660    False
119661    False
119662    False
          ...  
239333    False
239334    False
239335    False
239336    False
0         False
Name: Destination, Length: 358998, dtype: bool

In [103]:
searches_nd = searches_nd.drop([358997])

In [104]:
#Wrong lenght in the departure airport code in the searches file
wrong_len_s_dep = searches_nd["Origin"].str.len() < 3
wrong_len_s_dep.sort_values(ascending= False)

358996    False
119667    False
119659    False
119660    False
119661    False
          ...  
239332    False
239333    False
239334    False
239335    False
0         False
Name: Origin, Length: 358997, dtype: bool

In [105]:
np.count_nonzero(wrong_len_s_dep)

0

In [106]:
bookings_nd["arr_port"] = bookings_nd["arr_port"].str.upper()
bookings_nd["dep_port"] = bookings_nd["dep_port"].str.upper()

In [111]:
searches_nd["Date"] = searches_nd["Date"].str[0:10]
searches_nd["Date"]

0         2013-01-01
1         2013-01-01
2         2013-01-01
3         2013-01-01
4         2013-01-01
             ...    
358992    2013-12-25
358993    2013-12-25
358994    2013-12-25
358995    2013-12-25
358996    2013-01-01
Name: Date, Length: 358997, dtype: object

In [113]:
bookings_nd["act_date"] = bookings_nd["act_date"].str[0:10]
bookings_nd["act_date"]

0          2013-03-05
1          2013-03-26
2          2013-03-26
3          2013-03-26
4          2013-03-26
              ...    
999997     2013-05-28
999998     2013-05-28
999999     2013-05-28
1000000    2013-03-26
1000002    2013-03-25
Name: act_date, Length: 1000002, dtype: object

Now, I assume a search ended up being a booking if:
- Both the origin and destination airport is the same
- The date of the search is the same as the booking date

Let's add a columns with "1" in the bookings file, then I merge both datasets, I'll know that the ones with 1 ended up in fact being a booking

In [114]:
bookings_nd["Booked"] = 1

bookings_nd.head()

,act_date,dep_port,arr_port,Booked
0,2013-03-05,ZRH,LHR,1
1,2013-03-26,SAL,CLT,1
2,2013-03-26,SAL,CLT,1
3,2013-03-26,AKL,SVO,1
4,2013-03-26,AKL,SVO,1


In [117]:
searches_nd.drop(["Time"], axis = 1, inplace= True)

In [118]:
searches_bookings = searches_nd.merge(bookings_nd, how = "left", 
                                      left_on= ["Date", "Origin", "Destination"],
                                     right_on= ["act_date", "dep_port", "arr_port"])

In [119]:
searches_bookings.head()

,Date,Origin,Destination,act_date,dep_port,arr_port,Booked
0,2013-01-01,TXL,AUH,NaN,NaN,NaN,NaN
1,2013-01-01,ATH,MIL,NaN,NaN,NaN,NaN
2,2013-01-01,ICT,SFO,NaN,NaN,NaN,NaN
3,2013-01-01,RNB,ARN,NaN,NaN,NaN,NaN
4,2013-01-01,OSL,MAD,NaN,NaN,NaN,NaN


In [121]:
searches_bookings[searches_bookings["Booked"]==1].sample(10)

,Date,Origin,Destination,act_date,dep_port,arr_port,Booked
13588,2013-01-11,LHR,PVG,2013-01-11,LHR,PVG,1.0
160404,2013-05-21,ZRH,SAN,2013-05-21,ZRH,SAN,1.0
121360,2013-04-15,MUC,LHR,2013-04-15,MUC,LHR,1.0
187164,2013-06-16,DMM,JED,2013-06-16,DMM,JED,1.0
21340,2013-01-17,ZRH,HAM,2013-01-17,ZRH,HAM,1.0
9159,2013-01-08,BKK,CNX,2013-01-08,BKK,CNX,1.0
292344,2013-09-26,SIN,HKG,2013-09-26,SIN,HKG,1.0
18168,2013-01-15,SIN,MEL,2013-01-15,SIN,MEL,1.0
162965,2013-05-24,LIS,GIG,2013-05-24,LIS,GIG,1.0
100987,2013-03-27,MAD,PRG,2013-03-27,MAD,PRG,1.0


Let's delete the duplicated columns (with the same information)

In [122]:
searches_bookings.drop(["act_date", "dep_port", "arr_port"], axis = 1, inplace= True)

In [123]:
searches_bookings.head()

,Date,Origin,Destination,Booked
0,2013-01-01,TXL,AUH,NaN
1,2013-01-01,ATH,MIL,NaN
2,2013-01-01,ICT,SFO,NaN
3,2013-01-01,RNB,ARN,NaN
4,2013-01-01,OSL,MAD,NaN


Now, let's replace the NaN values with 0, so we know those searches didn't end up in a booking

In [124]:
searches_bookings["Booked"]= searches_bookings["Booked"].fillna(0)

In [125]:
searches_bookings.head()

,Date,Origin,Destination,Booked
0,2013-01-01,TXL,AUH,0.0
1,2013-01-01,ATH,MIL,0.0
2,2013-01-01,ICT,SFO,0.0
3,2013-01-01,RNB,ARN,0.0
4,2013-01-01,OSL,MAD,0.0


Now that we have what we wanted, let's see how many searches ended up in a booking:

In [127]:
bookings_1 = searches_bookings["Booked"] == 1
bookings_1

0         False
1         False
2         False
3         False
4         False
          ...  
386503    False
386504    False
386505    False
386506    False
386507    False
Name: Booked, Length: 386508, dtype: bool

In [128]:
np.count_nonzero(bookings_1)

45025

In [129]:
len(searches_bookings)

386508

In [131]:
(45025/386508)*100

11.649176731141399

## 11.65% of searches ended up in a booking! (This is 45.025 bookings)

Let's save this in a csv:

In [135]:
searches_bookings.to_csv("C:/Users/paula/Desktop/Paula/Máster/9. Data Science challenge/challenge/searches_bookings.csv.bz2", sep = "^", index = False)


In [147]:
ls

 El volumen de la unidad C es OS
 El número de serie del volumen es: 4CED-12B7

 Directorio de C:\Users\paula\Desktop\Paula\Máster\9. Data Science challenge\challenge

04/07/2021  19:09    <DIR>          .
04/07/2021  19:09    <DIR>          ..
12/06/2021  09:42       554.970.628 bookings.csv.bz2
04/07/2021  18:18        55.519.627 bookings_without_duplicates.csv.bz2
12/06/2021  09:42       483.188.920 searches.csv.bz2
04/07/2021  19:09         1.253.529 searches_bookings.csv.bz2
04/07/2021  17:07         2.046.353 searches_without_duplicates.csv.bz2
               5 archivos  1.096.979.057 bytes
               2 dirs  758.672.650.240 bytes libres
